### Importation of Necessary Libraries

In [ ]:
# -- imports OpenCV library for image and video processing

import cv2 

# -- imports imutils library for convenient image resizing and manipulation

import imutils

# -- imports the operating system library for file and directory operations

import os

# -- imports the time library for timing and delay-related operations

import time

# -- imports the datetime library for timestamping saved images and videos

from datetime import datetime

### Initial Setup and Configuration

In [ ]:
save_path = "detections" # directory to store captured cheating detections (images/videos)
os.makedirs(save_path, exist_ok=True) # creates the folder if it doesn't exist

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") # loads pre-trained Haar cascade for face detection

In [ ]:
cap = cv2.VideoCapture(0)  # initialize camera (0 = default webcam or CSI camera)

In [ ]:
# sets the desired frame dimensions for the video feed
frame_width = 640
frame_height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

In [ ]:
# defines a reference center point of the frame for face position comparison
center_x = frame_width // 2
center_y = frame_height // 2
margin = 50 # sets the threshold tolerance for detecting off-center movement


In [ ]:
# defines the flags and variables for video recording and timing
recording = False
record_start_time = None
cheating_start_time = None
out = None # videowriter object placeholder
print("Press 'q' to quit.")

### Main loop: Frame Capture and Processing

In [ ]:
while True:
    # captures a single frame from the camera
    ret, frame = cap.read()
    if not ret or frame is None:
        print(" Warning: Failed to grab frame.")
        continue

    # recalculates the frame center (in case of resolution changes)
    center_x = frame.shape[1] // 2
    center_y = frame.shape[0] // 2

    # resizes the frame to maintain consistent width and improve processing speed
    try:
        frame = imutils.resize(frame, width=frame_width)
    except Exception as e:
        print(f"Resize error: {e}")
        continue

    # converts the frame to grayscale for to reduce computational load and for faster face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detects faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    cheating_detected = False # flag to track cheating instances

    # THE FACE DETECTION AND BEHAVIOR ANALYSIS
    
    for (x, y, w, h) in faces:

        # computes the center of the detected face
        face_center_x = x + w // 2
        face_center_y = y + h // 2

        # calculates the displacement from the frame center
        dx = face_center_x - center_x
        dy = face_center_y - center_y

        # Determines the face posture/direction based on the position relative to the center
        if abs(dx) < margin and abs(dy) < margin:
            direction = "good sitting posture"
        elif abs(dx) > abs(dy):
            direction = "cheating" # horizontal movements likely indicate giraffing or looking sideways
        else:
            if dy > margin:
                direction = "good sitting posture"
            elif dy < -margin:
                direction = "warning" # suspicious upward movements
            else:
                direction = "good sitting posture"

        # Assigns bounding box colors for visualization
        if direction == "cheating":
            box_color = (0, 0, 255) # Red - (BGR from format)
            cheating_detected = True
        elif direction == "warning":
            box_color = (0, 255, 255) # Yellow - (BGR from format)
        else:
            box_color = (139, 0, 0) # Red - (BGR from format)

        # Draws bounding box & label on around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)
        cv2.putText(frame, direction, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, box_color, 2)

    # THE CHEATING DETECTION AND RECORDING
    
    if cheating_detected:
        
        # Starts timing the cheating instance
        if cheating_start_time is None:
            cheating_start_time = time.time()
        
        # Starts recording if cheating persists for atleast 2 seconds
        elif time.time() - cheating_start_time >= 2 and not recording:
            timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Captures a snapshot image with a timestamp
            screenshot_path = os.path.join(save_path, f"cheating_{timestamp_str}.jpeg")
            snapshot_frame = frame.copy()
            cv2.putText(snapshot_frame, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        (10, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.7, (255, 255, 255), 2)
            cv2.imwrite(screenshot_path, snapshot_frame)
            print(f"📸 Full-frame screenshot saved: {screenshot_path}")

            # Initializes Video Writer to record a short clip 
            video_path = os.path.join(save_path, f"cheating_{timestamp_str}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(video_path, fourcc, 20.0, (frame_width, frame_height))
            record_start_time = time.time()
            recording = True
            print(f"🎥 Recording full frame: {video_path}")
    else:
        cheating_start_time = None # resets if no suspicious activity was detected
        

    # RECORDING MANAGEMENT
    
    if recording:
        # Adds a timestamp overlay to each recorded frame
        frame_with_ts = frame.copy()
        cv2.putText(frame_with_ts, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    (10, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255, 255, 255), 2)
        
        # saves frame to the recording file
        out.write(frame_with_ts)

        # stops recording after 10 seconds
        if time.time() - record_start_time >= 10: 
            recording = False
            out.release()
            print("✅ Recording stopped after 10 seconds.")

    # DISPLAYS THE RESULTS IN REAL-TIME

    # uncomment the next line to visualize the frame center on the live feed
   # cv2.circle(frame, (center_x, center_y), 5, (0, 255, 255), -1)

    cv2.imshow("Face Tracker", frame)

    # exits the loop when the user presses "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

### Cleanup Operations

In [ ]:
cap.release() # releases the cameras
if out:
    out.release() # releases video capture of active
cv2.destroyAllWindows() # closes all OpenCV windows
